## Exploring Key Factors Influencing GitHub Repository Popularity

What factors significantly influence the popularity of a repository (i.e., the number of Stars)? Can we predict the number of Stars a repository receives using these factors?

In [2]:
#Load library
library(readr)
library(tidyverse)
library(rsample)
library(leaps)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.0.2
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [27]:
#Load dataset
repositories <- read_csv("repositories.csv")

Rows: 215029 Columns: 24
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): Name, Description, URL, Homepage, Language, License, Topics, Defau...
dbl  (5): Size, Stars, Forks, Issues, Watchers
lgl  (9): Has Issues, Has Projects, Has Downloads, Has Wiki, Has Pages, Has ...
dttm (2): Created At, Updated At

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Introduction

## Methods and Results

- Clean the dataset
- (prediction)split training&test
- forward selection
- fit the model
- assumptions??
- interpret their coefficients
- (prediction) may be cross-valiation on the training set
- (prediction) fit the model on the test set

FEEL FREE TO EDIT!!

In [10]:
#Remove rows with missing values
repo_tidy <- repositories %>%
    drop_na()
colnames(repo_tidy)

[1] "Name"            "Description"     "URL"             "Created At"     
 [5] "Updated At"      "Homepage"        "Size"            "Stars"          
 [9] "Forks"           "Issues"          "Watchers"        "Language"       
[13] "License"         "Topics"          "Has Issues"      "Has Projects"   
[17] "Has Downloads"   "Has Wiki"        "Has Pages"       "Has Discussions"
[21] "Is Fork"         "Is Archived"     "Is Template"     "Default Branch"

In [28]:
# !!! here is my code for data wrangling where i also change the binary variables into `factor` type, 
# and i remove some of the input variables

# i have not run this code so feel free to share your wrangling method and code
# or rewrite this part especially for the preliminary variable selection part!!
data_repo_clean <- repositories %>%
  mutate(across(c(Has.Issues, Has.Projects, Has.Downloads, Has.Wiki, Has.Pages, 
                  Has.Discussions, Is.Archived, Is.Template), as.factor)) %>%
  select(Size, Stars, Forks, Issues, Has.Issues, Has.Projects, Has.Downloads, Has.Wiki, Has.Pages, Has.Discussions, Is.Archived, Is.Template)

ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `across(...)`.
[1mCaused by error in `across()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `Has.Issues` doesn't exist.


In [8]:
#Split the dataset in 0.6
set.seed(123)

repo_split <- 
    repo_tidy %>%
    initial_split(prop = 0.6, strata = Stars)

training_repo <- training(repo_split)
testing_repo <- testing(repo_split)

In [ ]:
# head(training_repo)
repo_forward_sel <- regsubsets(
  x=Stars ~ ., 
  data=training_repo,
  method="backward",
)
summary()

## Discussion

## References

In [ ]:
# test